In [1]:
import pandas as pd

df = pd.read_csv('./dataset/sampled_clean_desc_only.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103688 entries, 0 to 103687
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PRODUCT_ID       103688 non-null  int64  
 1   TITLE            103688 non-null  object 
 2   BULLET_POINTS    103688 non-null  object 
 3   DESCRIPTION      103688 non-null  object 
 4   PRODUCT_TYPE_ID  103688 non-null  int64  
 5   PRODUCT_LENGTH   103688 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 4.7+ MB


1) description 병합

In [3]:
df["description"] = df["BULLET_POINTS"].astype(str) + "\n" + df["DESCRIPTION"].astype(str)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103688 entries, 0 to 103687
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PRODUCT_ID       103688 non-null  int64  
 1   TITLE            103688 non-null  object 
 2   BULLET_POINTS    103688 non-null  object 
 3   DESCRIPTION      103688 non-null  object 
 4   PRODUCT_TYPE_ID  103688 non-null  int64  
 5   PRODUCT_LENGTH   103688 non-null  float64
 6   description      103688 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 5.5+ MB


In [9]:
print("평균 description 길이:", df["description"].str.len().mean())
print("가장 긴 description 길이:",df["description"].str.len().max())
print("가장 짧은 description 길이:",df["description"].str.len().min())

평균 description 길이: 1189.7776502584677
가장 긴 description 길이: 6445
가장 짧은 description 길이: 3


In [10]:
MAX_LEN = 2000

df["description"] = df["description"].str.slice(0, MAX_LEN)

In [11]:
print("평균 description 길이:", df["description"].str.len().mean())
print("가장 긴 description 길이:",df["description"].str.len().max())
print("가장 짧은 description 길이:",df["description"].str.len().min())

평균 description 길이: 1090.6445876089808
가장 긴 description 길이: 2000
가장 짧은 description 길이: 3


In [12]:
min_len = 30
remove_too_short_data_df = df[df["description"].str.len() >= min_len]

In [13]:
remove_too_short_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103506 entries, 0 to 103687
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PRODUCT_ID       103506 non-null  int64  
 1   TITLE            103506 non-null  object 
 2   BULLET_POINTS    103506 non-null  object 
 3   DESCRIPTION      103506 non-null  object 
 4   PRODUCT_TYPE_ID  103506 non-null  int64  
 5   PRODUCT_LENGTH   103506 non-null  float64
 6   description      103506 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 6.3+ MB


In [31]:
df = remove_too_short_data_df

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103506 entries, 0 to 103687
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PRODUCT_ID       103506 non-null  int64  
 1   TITLE            103506 non-null  object 
 2   BULLET_POINTS    103506 non-null  object 
 3   DESCRIPTION      103506 non-null  object 
 4   PRODUCT_TYPE_ID  103506 non-null  int64  
 5   PRODUCT_LENGTH   103506 non-null  float64
 6   description      103506 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 6.3+ MB


In [33]:
import random

seed = pd.DataFrame()

# 1) 필드 매핑
seed["external_product_id"] = df["PRODUCT_ID"]
seed["name"] = df["TITLE"]
seed["description"] = df["DESCRIPTION"]
seed["product_type_id"] = df["PRODUCT_TYPE_ID"].astype(str)

# 2) category_key 생성 (일단 product_type 기반)
seed["category_key"] = seed["product_type_id"]

# 3) synthetic price 생성
def generate_price(_):
    return random.randint(5000, 150000)

seed["base_price"] = seed["category_key"].apply(generate_price)

print(seed.head())
seed.to_csv("./dataset/seed_products.csv", index=False)

   external_product_id                                               name  \
0              2130707  Tailgating Pros American Flag Whitewashed Rust...   
1              2746668  Venus Aqua Aquarium Air Pump Fishing Supplier ...   
2              1611407  Stylish Tempered Glass backsplash – Glass Kitc...   
3              2785306  TODO DEALS 9H Unbreakable Car Music System Scr...   
4              1032025                    Beck Arnley 041-0857 Oil Filter   

                                         description product_type_id  \
0  Tailgating Pros Cornhole Set w/ Bean Bags Incl...            1168   
1  <b><p>Venus Aqua Aquarium Air Pump Fishing Sup...            6995   
2  <p>This 4 mm thick toughened glass BACKSPLASH ...           12621   
3  ULTIMATE PROTECTION AND USABILITY: Our Premium...            2211   
4                             Steering Wheel Air Bag            7581   

  category_key  base_price  
0         1168       56890  
1         6995       81743  
2        12621   

In [34]:
seed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103506 entries, 0 to 103687
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   external_product_id  103506 non-null  int64 
 1   name                 103506 non-null  object
 2   description          103506 non-null  object
 3   product_type_id      103506 non-null  object
 4   category_key         103506 non-null  object
 5   base_price           103506 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 5.5+ MB


In [35]:
def resolve_template_key(row):
    title = row["name"].lower()
    ptype = row["product_type_id"]

    # PRODUCT_TYPE 기반 우선 매핑
    if ptype in PRODUCT_TYPE_TEMPLATE_MAP:
        return PRODUCT_TYPE_TEMPLATE_MAP[ptype]

    # TITLE 기반 규칙
    if any(x in title for x in ["t-shirt", "shirt", "top", "hoodie", "sleeve"]):
        return "FASHION_TOP"

    if any(x in title for x in ["pants", "jeans", "legging", "trouser", "shorts"]):
        return "FASHION_BOTTOM"

    if any(x in title for x in ["shoe", "sneaker", "boots", "slipper"]):
        return "SHOES"

    if any(x in title for x in ["curtain", "mat", "rug", "cushion", "pillow", "bedsheet"]):
        return "HOME_DECOR"

    if any(x in title for x in ["cream", "lotion", "serum", "oil", "shampoo", "perfume"]):
        return "BEAUTY"

    if any(x in title for x in ["charger", "adapter", "battery", "camera", "headphone"]):
        return "ELECTRONICS"

    if any(x in title for x in ["kitchen", "bottle", "cup", "pot", "pan", "storage"]):
        return "KITCHEN"

    if any(x in title for x in ["bracelet", "ring", "necklace", "earring", "strap"]):
        return "ACCESSORY"

    if any(x in title for x in ["book", "novel", "guide"]):
        return "BOOK"

    return "DEFAULT"

seed["template_key"] = seed.apply(resolve_template_key, axis=1)
seed.to_csv("seed_products_with_template.csv", index=False)
print(seed["template_key"].value_counts())

template_key
DEFAULT           60526
FASHION_TOP       11175
HOME_DECOR         8918
KITCHEN            7610
ACCESSORY          4471
SHOES              3888
ELECTRONICS        2170
BEAUTY             2163
FASHION_BOTTOM     1788
BOOK                797
Name: count, dtype: int64


In [38]:
import itertools
import uuid

OPTION_TEMPLATE = {
    "FASHION_TOP": {
        "Color": ["Black", "White", "Beige", "Navy"],
        "Size": ["S", "M", "L", "XL"]
    },
    "FASHION_BOTTOM": {
        "Color": ["Black", "Navy", "Gray"],
        "Size": ["S", "M", "L", "XL"]
    },
    "SHOES": {
        "Color": ["White", "Black"],
        "Size": ["230", "240", "250", "260", "270", "280"]
    },
    "HOME_DECOR": {
        "Color": ["Gray", "Beige", "Navy"],
        "Size": ["140x230", "160x260"]
    },
    "BEAUTY": {
        "Volume": ["30ml", "50ml", "100ml"],
        "Package": ["Single", "2-Pack"]
    },
    "ELECTRONICS": {
        "Model": ["Standard", "Pro", "Max"],
        "Color": ["Black", "White"]
    },
    "KITCHEN": {
        "Color": ["Black", "Silver", "White"],
        "Capacity": ["Small", "Medium", "Large"]
    },
    "ACCESSORY": {
        "Color": ["Black", "Brown", "Gray"],
        "Style": ["Classic", "Modern"]
    },
    "BOOK": {},
    "DEFAULT": {},
}

sku_rows = []

for _, row in seed.iterrows():
    template = OPTION_TEMPLATE[row["template_key"]]

    # 옵션이 없는 상품
    if not template:
        sku_rows.append({
            "sku_id": str(uuid.uuid4()),
            "external_product_id": row["external_product_id"],
            "price": row["base_price"],
            "options": {},
        })
        continue

    # 옵션 조합 SKU 생성
    keys = list(template.keys())
    values = list(template.values())

    for comb in itertools.product(*values):
        sku_rows.append({
            "sku_id": str(uuid.uuid4()),
            "external_product_id": row["external_product_id"],
            "price": row["base_price"] + random.randint(-3000, 3000),
            "options": dict(zip(keys, comb))
        })

sku_df = pd.DataFrame(sku_rows)
sku_df.to_csv("./dataset/sku.csv", index=False)
print("SKU 생성 개수:", len(sku_df))

SKU 생성 개수: 483057


In [39]:
sku_df["stock"] = sku_df["sku_id"].apply(lambda _: random.randint(0, 500))
sku_df.to_csv("./dataset/sku_with_stock.csv", index=False)

In [40]:
sku_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483057 entries, 0 to 483056
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sku_id               483057 non-null  object
 1   external_product_id  483057 non-null  int64 
 2   price                483057 non-null  int64 
 3   options              483057 non-null  object
 4   stock                483057 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 18.4+ MB
